<a href="https://colab.research.google.com/gist/shreykshah/094b83eca043d96b08f8e9dc49b8ba17/class_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook was desgined to be run in Google Colab. You can find a copy by clicking the button above. To use locally, ensure all packages (!pip install ...) are installed and ignore all errors caused by !commands.

# inputs

Enter courses in the format `("course_name", course_credits, [potential_semesters], [prerequisites]`. Ensure the course names and semster names do not have a "`_`" (underscore) character. This is not checked, but will cause the program to fail using *z3*.

Enter the semesters in the format `(semester_name, max_credits_this_semester)`. All semesters that exist in the courses list must be in the semesters list. Put the semesters in order if listing prerequisites.

*z3* generally runs slightly faster than *max flow* and has greater functionality (ie. nonsingular courses). *augmented z3* is the only function that works with prerequisites, but does tend to run longer.

In [ ]:
courses = []
courses.append(("course name", 1, ["semester 1", 2, "test_sem_name"]))
              # course name, credits, semesters
courses.append(("course2", 1, [2, "test_sem_name"], ["course name"]))
              # course name, credits, semesters,    prerequisites

sems = []
sems.append(("semester 1", 3))
            #semester name, max credits to be taken this semester
!pip install requests
import requests
r = requests.get('https://pages.cs.wisc.edu/~shrey/cc/cclassschedulerc')
#please leave this in if forked
print(courses, sems, sep='\n')

### Check inputs

In [ ]:
check_sems = set()
for course in courses:
  for sem_option in course[2]:
    check_sems.add(sem_option)
for sem in sems:
  check_sems.discard(sem[0])
if len(check_sems) != 0:
  raise ValueError('missing semester definition in list "sems"') 

In [99]:
if sum([weight for _, weight, _, _ in courses]) > sum([max_ssize for _, max_ssize in in_set_list]):
  raise ValueError('total course credits exceed max possible semester credits') 

# z3

## initialize

In [ ]:
!pip install z3-solver
from z3 import *

In [ ]:
s = Solver()

## regular z3

In [ ]:
semesters = {sem[0]: [] for sem in sems}

In [ ]:
#course mappings
for course in courses:
  course_list = []
  for sem in course[2]:
    course_list.append(If(Bool(str(course[0]) + "_" + str(sem)), course[1], 0))
    semesters[sem].append(If(Bool(str(course[0])+"_"+str(sem)), course[1], 0))
  s.add(Sum(course_list) == course[1])

In [ ]:
#semester mappings
for sem in sems:
  s.add(Sum(semesters[sem[0]]) <= sem[1])

In [ ]:
try:
  print(s.check())
  res = s.model()
except Z3Exception:
  raise ValueError('unsat - too many constraints, cannot fit all courses as given')

In [ ]:
rets = {str(sem[0]): [] for sem in sems}
for x in res:
  if res[x] == True:
    pair = str(x).split('_')
    rets[pair[1]].append(pair[0])

## augmented z3

In [ ]:
v = {varname: Int(varname) for varname, *_ in courses}

In [ ]:
courses = [tup if len(tup) == 4 else (*tup, []) for tup in courses]

In [ ]:
for name, weight, pos_sets, prereq in courses:
    s.add(Or([v[name] == p for p in pos_sets])) # each var should be in one of its possible sets
    s.add(And([v[r] < v[name] for r in prereq])) # each prerequisit should be in an earlier set

In [ ]:
for snum, max_ssize in sems:
    s.add(Sum([If(v[name] == snum, weight, 0) for name, weight, _, _ in courses]) <= max_ssize)

In [ ]:
res = s.check()
print(res)
if res == sat:
    m = s.model()

In [ ]:
set_assignments = {name: m.evaluate(v[name]).as_long() for name, *_ in courses}
# print("assignments:", set_assignments)
for snum, desired_ssize in in_set_list:
    ssize = sum([weight for name, weight, _, _ in courses if set_assignments[name] == snum])
    print(f"({ssize}/{desired_ssize}) {snum}:", [name for name, *_ in courses if set_assignments[name] == snum])

# max flow

In [ ]:
!pip install ortools

In [ ]:
from ortools.graph import pywrapgraph

In [ ]:
start_nodes = []
end_nodes = []
capacities = []

In [ ]:
i = 2
j = len(courses) + 2
indexed = dict()

In [ ]:
for course in courses:
  # reject duplicate courses
  if course[0] in indexed:
    raise ValueException("repeated course listing '" + course[0] + "'")
  #double index course
  indexed[str(course[0])] = i
  indexed[i] = course[0]

  for sem in course[2]:
    #double index semester
    if str(sem) not in indexed:
      indexed[str(sem)] = j
      indexed[j] = str(sem)
      j += 1
    #course to semester
    start_nodes.append(i)
    end_nodes.append(indexed[str(sem)])
    capacities.append(course[1])
  #source to course
  start_nodes.append(0)
  end_nodes.append(i)
  capacities.append(course[1])
  
  i += 1

In [ ]:
#semester to sink
for sem_indv in sems:
  start_nodes.append(indexed[str(sem_indv[0])])
  end_nodes.append(1)
  capacities.append(sem_indv[1])

print(start_nodes, end_nodes, capacities)

In [ ]:
max_flow = pywrapgraph.SimpleMaxFlow()
# Add each arc.
for i in range(0, len(start_nodes)):
  max_flow.AddArcWithCapacity(start_nodes[i], end_nodes[i], capacities[i])

In [ ]:
rets = dict()
# Find the maximum flow
if max_flow.Solve(0, 1) == max_flow.OPTIMAL:
  for i in range(max_flow.NumArcs()):
    if max_flow.Tail(i) > 1 and max_flow.Head(i) > 1 and max_flow.Flow(i) > 0:
      if str(indexed[max_flow.Head(i)]) not in rets:
        rets[indexed[max_flow.Head(i)]] = []
      rets[indexed[max_flow.Head(i)]].append(indexed[max_flow.Tail(i)])
  print(rets)
else:
  print('There was an issue with the max flow input.')

# print results

In [ ]:
#for regular z3 and max flow
#  to see augmented z3 output, please go directly to the final cell in az3 
from pprint import pprint
pprint(rets)